# Python Performance for Poets

## Gil Forsyth and Steven Lott
### Capital One

# Who are you and why should we listen to you?

## Gil Forsyth
##### github: gforsyth

#### Data Scientist at Capital One
#### Core contributor and maintainer of `xonsh`
#### Recovering PhD student in HPC / computational fluid dynamics
#### SciPy Program Chair

# Profiling tools introduction

"Premature optimization is the root of all evil" - Donald Knuth

"Undirected optimization is just kind of pointless, I think" - Gil Forsyth

Profilers are timing tools.  They help you identify, at different levels of granularity, why your program takes time $T \pm \Delta t$ to run.

You can use the information from a profile to make efficient use of _your_ time.  If you spend a day making `function_a` run in 2 ms instead of 5 ms, that's great, except that `function_b` still takes 3 minutes to complete.

The tools used in this notebook are for use in... the notebook.  They do, however, all have command-line or REPL equivalents which are noted below each example.

## Start with a poorly written program

There is a lot wrong with this program, starting with the fact that it doesn't do anything.  Let's ignore that for a moment and get a feel for how long it takes to run.

In [ ]:
import math
from time import sleep

def weird_call(S, K, T, r, sigma):
    """
    S: spot price
    K: strike price
    T: time to maturity
    r: interest rate
    sigma: volatility of underlying asset
    """
    def cdf(x):
        # CDF function for the standard normal distribution
        return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0
    
    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = (math.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    
    call = (S * cdf(d1) - K * math.exp(-r * T) * cdf(d2))
    
    return call

def bad_call(dude):
    sleep(.1)
    
def worse_call(dude):
    sleep(.5)
    
def sumulate(foo):
    if not isinstance(foo, int):
        return
    
    ans = 0
    for i in range(foo):
        ans += i
        
    weird_call(50, 100, 1, 0.05, 0.25)
    bad_call(ans)
    worse_call(ans)
        
    return ans

# `time`

In [ ]:
%%time 
sumulate(150)

real / total: amount of CPU time spent

user: amount of CPU time spent in user mode.

system / sys:  amount of CPU time spent in kernel mode

wall: the time from start to finish of the call. Literally how much time has elapsed on the clock on the wall.

A couple of things to note:

* `sumulate(150)` was run only once
* The output of that run was returned to us

### Command line equivalent (on OS X and Linux)

Use the program `time`

```console
time python -c "from poorly_written import sumulate; sumulate(150)"
```


# `timeit`

That's a start, but computers have a lot of other... stuff... running.  What if I ran `%time` while Dropbox was having one a little fit?  Or Docker Desktop was trying to inhale all of my CPU cores?

It would be better if we could run it a bunch of times and then look at some aggregate results, right?

That's what `timeit` is for.

In [ ]:
%timeit sumulate(150)

Wow, so that took much longer than the reported time to finish.  Why?  Because it ran `sumulate(150)` in a loop with 1 iteration, 7 separate times.

`timeit` has a few heuristics to balance how long it takes to run with getting a useful number of results.

What does that mean?  Let's try it out on something that is faster than `sumulate`

In [ ]:
%timeit 1 + 1

For a simple addition, `timeit` ran 7 separate times, with 100 million loops in each run.  It just ran that addition 700 _million_ times.  

All of these options are configurable:

In [ ]:
%timeit?

The `-o` option can be especially useful if you want to save results or perform additional checks on them:

In [ ]:
results = %timeit -o -q sumulate(150)

In [ ]:
results.all_runs

In [ ]:
results.loops

#### Pre-run setup

If you need to run an import or set a value for a `timeit` run but don't want that part of the code to be timed, you can list it in the first line of the multi-line `%%timeit` magic:

In [ ]:
%%timeit import re; RE_PUNC=re.compile("[^a-z/s]")  # this isn't part of the timing
re.sub(RE_PUNC, " ", "no.more.punctuation")

In [ ]:
%%timeit
import re
RE_PUNC=re.compile("[^a-z/s]")
re.sub(RE_PUNC, " ", "no.more.punctuation")

### Command line equivalent

To run `timeit` in a terminal, call the `timeit` module:

```console
python -m timeit "1 + 1"
```

Running `timeit` this way is when we are more likely to need to do some setup (since in the notebook functions are usually already defined):

```console
python -m timeit -s "import re; RE_PUNC=re.compile('[^a-z/s]')" "re.sub(RE_PUNC, ' ', 'no.more.punctuation')"
```

To see all the available options:

```console
python -m timeit -h
```

# `prun` and `cProfile`

`time` and `timeit` return overall runtimes but don't give us any more granularity into which parts of a program are taking up time.

Remember, `sumulate` is calling three separate functions, how can we tell which of them is taking up which portion of the overall run time?

We use a profiler!

In the notebook and IPython, we can use the `prun` magic to profile a function:

In [ ]:
%prun sumulate(150)

In [ ]:
%prun?


`tottime`: The total time spent in this function excluding functions called by this function.

`cumtime`: The cumulative time spent in this function including the time spent in all functions called in this function.

`ncalls`: The total number of calls to the function.


Alternatively, you can import the `cProfile` module and use it directly. 
By default it returns a report like the one above.  Or you can save the results to a file for use with a separate visualization tool.

In [ ]:
import cProfile

In [ ]:
cProfile.run("sumulate(150)")

### Command line equivalent
```
python -m cProfile -o program.prof my_program.py
```

## Visualizing profiler output

In [ ]:
%load_ext snakeviz

In [ ]:
%snakeviz?

In [ ]:
%snakeviz sumulate(150)

### Command line equivalent

```console
python -m cProfile -o program.prof my_program.py
snakeviz program.prof
```


# `line_profiler`

`time` and `timeit` gave us overall timings.
`cProfile` gave us function-by-function timings.
What if we want to go further?  

Enter the `line_profiler`:

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f sumulate sumulate(150)
# -f <function to profile> <some call that will cause that function to run>

Let's save those results out to a text file and then load them for easier viewing:

In [ ]:
%lprun -T sumulate_lines.txt -f sumulate sumulate(150)

In [ ]:
%load sumulate_lines.txt

You can profile multiple functions in a single run:

In [ ]:
%lprun -f weird_call -f bad_call -f worse_call sumulate(150)